In [ ]:
time_difference = pd.to_datetime(fitbit_survey['enddate']) - fitbit_survey['startdate']
mean_total_days = np.mean(time_difference).days
columns = final_data_frame.columns
add_column = list(fitbit_survey.columns[-22:])
add_column.append('Id')

suffix_range_rest = [f"_{i}" for i in range(mean_total_days)]
set_all_columns_the_same = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range_rest)]

reorganize_data = pd.DataFrame()


In [ ]:
num_i = len(time_difference)

for i in range(num_i):
    i = i
    difference_i = time_difference[i]
    n_days = time_difference[i].days

    if n_days <= mean_total_days:
        suffix_range = [f"_{i}" for i in range(mean_total_days)]
        select_fitbit_columns = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range)]
        used_data = pd.DataFrame(fitbit_survey.iloc[i]).T[select_fitbit_columns]

    else:
        nan_day = []
        suffix_range_new = [f"_{i}" for i in range(mean_total_days+1, n_days)]

        for suffixs in suffix_range_new:
            suffix_columns = [col for col in columns if col.endswith(suffixs)]
            
            #print(len(suffix_columns))
            suffix_data = fitbit_survey.iloc[i][suffix_columns]

            if suffix_data.isna().all():
                #print(True)
                nan_day.append(suffixs)

        #select data after 65
        filter_suffix_range = [item for item in suffix_range_new if item not in nan_day]

        if len(filter_suffix_range) >= mean_total_days:
            new_filter_suffix_range = filter_suffix_range[-mean_total_days:]
            columns_65 = [col for col in columns if any(col.endswith(suffix) for suffix in new_filter_suffix_range)]
            columns_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[columns_65]
            used_data = columns_65_data

        else:
            #select data before 65
            #number of columns before 65
            n_col = mean_total_days-len(filter_suffix_range)
            start_number = mean_total_days-n_col
            end_number = mean_total_days

            suffix_range = [f"_{i}" for i in range(start_number+1, end_number+1)]
            select_fitbit_columns = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range)]
            before_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[select_fitbit_columns]

            #select dat aafter 65

            after_65_columns = [col for col in columns if any(col.endswith(suffix) for suffix in filter_suffix_range)]
            after_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[after_65_columns]

            used_data = pd.concat([before_65_data, after_65_data], axis = 1) #combine two

        used_data.columns = set_all_columns_the_same

    #append survey data
    survey_data = pd.DataFrame(fitbit_survey.iloc[i]).T[add_column]

    new_fitbit_survey = pd.concat([used_data, survey_data], axis = 1)
        
    reorganize_data = pd.concat([reorganize_data, new_fitbit_survey], axis = 0, ignore_index=True) #

Convert horizontal to vertical

In [ ]:
#convert horizontal to vertical
fitbit_survey_transfer = fitbit_survey.copy()
final_data_frame = pd.DataFrame()

for i1 in range(len(all_fitbit_id)):
    fitbit_id = all_fitbit_id[i1]
    select_data = fitbit_survey_transfer[fitbit_survey_transfer['Id'] == fitbit_id]

    len_select_data = len(select_data)

    process_series = pd.Series(dtype = 'object')

    for i2 in range(len_select_data):
        row_select_data = select_data.iloc[i2].copy()
        row_select_data.index = [str(col) + '_' + str(i2) for col in row_select_data.index] #revise the name
        process_series = pd.concat([process_series, row_select_data])

    trans_process_series = pd.DataFrame(process_series).T

    final_data_frame = pd.concat([final_data_frame, trans_process_series], axis = 0, ignore_index=True)

final_data_frame = final_data_frame.set_index(['Id_0'])#set ID and date as index
final_data_frame = final_data_frame.drop(final_data_frame.filter(regex = 'Id').columns, axis = 1) #delete column with the same format
final_data_frame.reset_index(inplace=True) #reset_index
final_data_frame.rename(columns = {'Id_0':'Id'}, inplace = True) #renamei d

In [ ]:
#combine with mental health dataset
mental_health_data = select_survey[['Id','mental_health_1','mental_health_2']]
final_data_mental_health = pd.merge(final_data_frame, mental_health_data, on = 'Id', how = 'inner')

In [ ]:
final_data_mental_health.to_csv('final_data_mental_health.csv') #mental health

In [ ]:

#convert horizontal to vertical
fitbit_survey_transfer = fitbit_survey.copy()
final_data_frame = pd.DataFrame()

for i1 in range(len(all_fitbit_id)):
    fitbit_id = all_fitbit_id[i1]
    select_data = fitbit_survey_transfer[fitbit_survey_transfer['Id'] == fitbit_id]

    len_select_data = len(select_data)

    process_series = pd.Series(dtype = 'object')

    for i2 in range(len_select_data):
        row_select_data = select_data.iloc[i2].copy()
        row_select_data.index = [str(col) + '_' + str(i2) for col in row_select_data.index] #revise the name
        process_series = pd.concat([process_series, row_select_data])

    trans_process_series = pd.DataFrame(process_series).T

    final_data_frame = pd.concat([final_data_frame, trans_process_series], axis = 0, ignore_index=True)

final_data_frame = final_data_frame.set_index(['Id_0'])#set ID and date as index
final_data_frame = final_data_frame.drop(final_data_frame.filter(regex = 'Id').columns, axis = 1) #delete column with the same format
final_data_frame.reset_index(inplace=True) #reset_index
final_data_frame.rename(columns = {'Id_0':'Id'}, inplace = True) #renamei d